In [1]:
import tangos
import pynbody as pnb
import pylab as p
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sys
from astropy.io import ascii
import scipy.constants as c
import argparse as ap
from sklearn.neighbors import KDTree
from datetime import datetime
import time
import pandas as pd
#import networkx as nx

In [3]:
snapshots = ['000098','000105','000111','000118','000126','000134','000142','000151','000160','000170','000181',\
            '000192','000204','000216','000229','000243','000256','000258','000274','000290','000308',\
            '000372','000437','000446',\
            '000463','000491','000512','000520','000547','000551','000584','000618','000655',\
            '000690','000694','000735','000768','000778','000824','000873','000909','000924','000979','001024',\
            '001036','001065','001097','001162','001230','001270','001280',\
            '001302','001378','001458','001536','001543','001550','001632','001726','001792','001826','001931','001945',\
            '002042','002048','002159','002281','002304','002411','002536','002547','002560','002690','002816','002840',\
            '002998','003072','003163','003328','003336','003478','003517','003584','003707','003840','003905','004096',\
            '004111','004173','004326','004352','004549','004608','004781','004864','005022','005107','005120','005271',\
            '005376','005529','005632','005795','005888','006069','006144','006350','006390','006400','006640','006656',\
            '006912','006937','007168','007212','007241','007394','007424','007552','007680','007779','007869','007936',\
            '008192']

#z2:snap001945:ID62
#z=1.79:snap002159:ID65
#z=1.5:snap002536:ID69
#z=1.22:snap002998:ID75
#z=1:snap003478:ID80
#z=0.75:snap004173:ID88
#z=0.7:snap004326:ID89
#z=0.5:snap005107:ID96
#z=0.25:snap006656:ID107
#z=0.21:snap006656:ID110
#z=0.05:snap007779:ID120
#AllIDs = 62,65,69,75,80,88,96,107,120]

IDs = [62,65,69,75,80,88,89,96,107,110,120]

################################################################## 

for i in IDs:
    ID = i
    print('ID',ID)
    snap = str(snapshots[ID])
    print('snap',snap)
    lastsnap = str(snapshots[ID-1])
    
        #get BHs infor that is saved from tangos
    hf = h5py.File('/home/vidasz/projects/rrg-babul-ad/vidasz/DualAGN/Datasets/HaloBH-TangosPynbodyMap-R25-snap{0}.hdf5'.format(snap), 'r')
    print(hf.keys())

    tngBHIDs = np.array(hf.get('BH_id'))
    tngBHmdot = np.array(hf.get('BH_mdot'))
    tngBHmass = np.array(hf.get('BH_mass'))
    tngHalo = np.array(hf.get('tangos_haloid'))
    pnbHalo = np.array(hf.get('pynbody_haloid'))

    hf.close()
    ####################################################################################################
    print(len(tngBHIDs))
    print(len(tngBHIDs))
    
    tangosData = {'BH_id':  tngBHIDs,
              'BH_mdot':  tngBHmdot,
              'BH_mass':  tngBHmass,
              'tangos_haloid':  tngHalo,
              'pynbody_haloid':  pnbHalo
             }

    df_tangos = pd.DataFrame(tangosData)

    print(len(df_tangos))
    print(len(df_tangos))
    
    # sepration threshold for dual AGNs
    sepration_threshold = 30 #kpc
    lum_thershold = 1.89e-3 # Luminosity > 1e43 and L_HXR > 1e42 erg/s = mdot > 1.89e-3
    BBHmass_threshold = 1e8

    # loading snapshots to access particles info
    data_file = '/home/vidasz/projects/rrg-babul-ad/SHARED/Romulus/cosmo25/cosmo25p.768sg1bwK1BHe75.{0}'.format(snap)
    s = pnb.load(data_file)
    s.physical_units()
    h = s.halos(ahf_mpi = True)
    age = pnb.analysis.cosmology.age(s, z=None, unit='Gyr')
    t_analysis = str(round(age, 3))
    name = s.properties['z']
    Redshift = str(round(name, 3))

    #pynbody BHs
    #get all BHs in simulation
    BH_idx = np.where(s.s['tform'] < 0)
    BH_iordssim = s.s['iord'][BH_idx]
    idx = np.isin(BH_iordssim,tngBHIDs)

    BH_iord = s.s['iord'][BH_idx][idx]
    BH_mass = s.s['mass'][BH_idx][idx]
    BH_vel = s.s['vel'][BH_idx][idx]
    BH_pos = s.s['pos'][BH_idx][idx].in_units('kpc')

    BH_pos_x = BH_pos[:,0]
    BH_pos_y = BH_pos[:,1]
    BH_pos_z = BH_pos[:,2]

    BH_vel_x = BH_vel[:,0]
    BH_vel_y = BH_vel[:,1]
    BH_vel_z = BH_vel[:,2]

    pynbodyData = {'BH_id':  BH_iord,
                   'BH_vx':  BH_vel_x,
                   'BH_vy':  BH_vel_y,
                   'BH_vz':  BH_vel_z,
                   'BH_x':  BH_pos_x,
                   'BH_y':  BH_pos_y,
                   'BH_z':  BH_pos_z
                  }

    df_pynbody = pd.DataFrame(pynbodyData)
    
    print(len(df_pynbody),len(df_tangos))
    
    merged_df_whalo0 = pd.merge(df_tangos, df_pynbody, on='BH_id', how='inner')
    merged_df = merged_df_whalo0[merged_df_whalo0['pynbody_haloid'] != 0]
    print('len merged_df_whalo0',len(merged_df_whalo0),'len merged_df', len(merged_df))
    
    # Step 1: Filter main black holes based on mdot threshold
    main_black_holes = merged_df[merged_df['BH_mdot'] > lum_thershold]

    # Step 2: Find neighboring black holes for each main black hole
    new_dataset = pd.DataFrame(columns=['BH_id', 'BH_mass', 'BH_vx', 'BH_vy', 'BH_vz', 'BH_x', 'BH_y', 'BH_z',
                                        'BH_mdot', 'tangos_haloid', 'pynbody_haloid', 'NeighbourBH_ID',
                                        'NeighbourBH_mass', 'NeighbourBH_vx', 'NeighbourBH_vy', 'NeighbourBH_vz',
                                        'NeighbourBH_x', 'NeighbourBH_y', 'NeighbourBH_z', 'NeighbourBH_mdot',
                                        'NeighbourBH_pynbody_haloid', 'NeighbourBH_tangos_haloid', 'NeighbourBH_counts'])

    for index, main_bh in main_black_holes.iterrows():
        main_bh_pos = [main_bh['BH_x'], main_bh['BH_y'], main_bh['BH_z']]
        distances = np.sqrt(np.sum(np.square(merged_df[['BH_x', 'BH_y', 'BH_z']].values - main_bh_pos), axis=1))
        neighbors = merged_df[(distances < sepration_threshold) & (merged_df['BH_mdot'] > lum_thershold)]
        neighbors = neighbors[neighbors['BH_id'] != main_bh['BH_id']]  # Exclude the main black hole itself

        main_info = main_bh[['BH_id', 'BH_mass', 'BH_vx', 'BH_vy', 'BH_vz', 'BH_x', 'BH_y', 'BH_z', 'BH_mdot',
                             'tangos_haloid', 'pynbody_haloid']].to_dict()

        neighbor_info = neighbors[['BH_id', 'BH_mass', 'BH_vx', 'BH_vy', 'BH_vz', 'BH_x', 'BH_y', 'BH_z',
                                   'BH_mdot', 'pynbody_haloid', 'tangos_haloid']]

        new_row = main_info.copy()
        new_row.update({
            'NeighbourBH_ID': neighbor_info['BH_id'].tolist(),
            'NeighbourBH_mass': neighbor_info['BH_mass'].tolist(),
            'NeighbourBH_vx': neighbor_info['BH_vx'].tolist(),
            'NeighbourBH_vy': neighbor_info['BH_vy'].tolist(),
            'NeighbourBH_vz': neighbor_info['BH_vz'].tolist(),
            'NeighbourBH_x': neighbor_info['BH_x'].tolist(),
            'NeighbourBH_y': neighbor_info['BH_y'].tolist(),
            'NeighbourBH_z': neighbor_info['BH_z'].tolist(),
            'NeighbourBH_mdot': neighbor_info['BH_mdot'].tolist(),
            'NeighbourBH_pynbody_haloid': neighbor_info['pynbody_haloid'].tolist(),
            'NeighbourBH_tangos_haloid': neighbor_info['tangos_haloid'].tolist(),
            'NeighbourBH_counts': len(neighbor_info)
        })

        new_dataset = new_dataset.append(new_row, ignore_index=True)

    print(len(new_dataset) == len(main_black_holes))
    print(len(new_dataset))
        
        # reading host props data set
    halo_df = pd.read_pickle('/home/vidasz/projects/rrg-babul-ad/vidasz/DualAGN/Datasets/AGNLum/Xray/L1e42/Halosproperties-R25-snap{0}.pkl'.format(snap))


# Merge two dataframes on ID columns
    merged_df = pd.merge(new_dataset, halo_df, how='inner', left_on='pynbody_haloid', right_on='Halo_ids')
    merged_df.to_pickle('/home/vidasz/projects/rrg-babul-ad/vidasz/DualAGN/Datasets/AGNLum/Xray/L1e42/Fulldf-LumNeighbours-R25-snap{0}.pkl'.format(snap,Redshift))
    #new_dataset.to_pickle('/home/vidasz/projects/rrg-babul-ad/vidasz/DualAGN/Datasets/AGNLum/Xray/L1e42/onlyBHs/onlyBH-LumNeighbours-R25-snap{0}.pkl'.format(snap,Redshift))
    
    print('len merged_df',len(merged_df), 'len new_dataset',len(new_dataset))
    

ID 62
snap 001945
<KeysViewHDF5 ['BH_id', 'BH_mass', 'BH_mdot', 'pynbody_haloid', 'tangos_haloid']>
5325
5325
5325
5325
5325 5325
len merged_df_whalo0 5325 len merged_df 5257
True
126
len merged_df 126 len new_dataset 126
ID 65
snap 002159
<KeysViewHDF5 ['BH_id', 'BH_mass', 'BH_mdot', 'pynbody_haloid', 'tangos_haloid']>
5221
5221
5221
5221
5221 5221
len merged_df_whalo0 5221 len merged_df 5114
True
150
len merged_df 150 len new_dataset 150
ID 69
snap 002536
<KeysViewHDF5 ['BH_id', 'BH_mass', 'BH_mdot', 'pynbody_haloid', 'tangos_haloid']>
5088
5088
5088
5088
5088 5088
len merged_df_whalo0 5088 len merged_df 5026
True
134
len merged_df 134 len new_dataset 134
ID 75
snap 002998
<KeysViewHDF5 ['BH_id', 'BH_mass', 'BH_mdot', 'pynbody_haloid', 'tangos_haloid']>
4957
4957
4957
4957
4957 4957
len merged_df_whalo0 4957 len merged_df 4901
True
107
len merged_df 107 len new_dataset 107
ID 80
snap 003478
<KeysViewHDF5 ['BH_id', 'BH_mass', 'BH_mdot', 'pynbody_haloid', 'tangos_haloid']>
4859
4859
48